In [ ]:
!pip install yfinance
!pip install tensorflow
!pip install scikit-learn

In [16]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split

# Tüpraş verisini indir
symbol = "TUPRS.IS"
start_date = "2020-11-27"
end_date = "2024-11-27"
data = yf.download(symbol, start=start_date, end=end_date)

# Veriyi sadece 'Close' (Kapanış Fiyatı) kullanarak model için hazırlayalım
data = data[['Close']]

# Min-Max scaling
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Veri setini eğitim ve test için bölelim
def create_dataset(data, time_step=60):
    X, y = [], []
    for i in range(time_step, len(data)):
        X.append(data[i-time_step:i, 0])  # X için son 60 günü al
        y.append(data[i, 0])  # y için o günün kapanış fiyatını al
    return np.array(X), np.array(y)

time_step = 60  # Son 60 günü baz alarak tahmin yapacağız
X, y = create_dataset(scaled_data)

# Eğitim ve test verilerini ayıralım
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Veriyi LSTM modeline uygun şekilde şekillendirelim
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# LSTM modelini oluştur
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=1))  # Sonuçta tek bir tahmin yapacağız

model.compile(optimizer='adam', loss='mean_squared_error')

# Modeli eğit
model.fit(X_train, y_train, epochs=10, batch_size=32)

[*********************100%***********************]  1 of 1 completed

Epoch 1/10



/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


24/24 ━━━━━━━━━━━━━━━━━━━━ 4s 57ms/step - loss: 0.0707
Epoch 2/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - loss: 0.0025
Epoch 3/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - loss: 0.0010
Epoch 4/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - loss: 6.5146e-04
Epoch 5/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 83ms/step - loss: 7.2027e-04
Epoch 6/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - loss: 6.7187e-04
Epoch 7/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - loss: 6.3417e-04
Epoch 8/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 54ms/step - loss: 6.9580e-04
Epoch 9/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - loss: 8.1689e-04
Epoch 10/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - loss: 7.2296e-04


In [19]:
# Test verisinin başlangıç tarihini daha geriye çek
test_start_date = "2024-09-01"  # Yeterli veri olması için daha erken tarih
test_end_date = "2024-11-27"   # Güncel tarih

# Veriyi indir
test_data = yf.download(symbol, start=test_start_date, end=test_end_date)
test_data = test_data[['Close']]

# Test verisini ölçekle
scaled_test_data = scaler.transform(test_data)

# X_test_new oluştur
X_test_new = []
for i in range(time_step, len(scaled_test_data)):
    X_test_new.append(scaled_test_data[i-time_step:i, 0])

X_test_new = np.array(X_test_new)

# Veriyi reshape yap
if X_test_new.shape[0] > 0:
    X_test_new = X_test_new.reshape(X_test_new.shape[0], X_test_new.shape[1], 1)
    predictions = model.predict(X_test_new)
    predictions = scaler.inverse_transform(predictions)
    print("Tahminler:", predictions)
else:
    print("Test verisi oluşturulamadı.")

[*********************100%***********************]  1 of 1 completed

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Tahminler: [[150.94804]]


In [20]:
# Son haftalık veriler üzerinde test yap ve karar ver
predictions = []
decisions = []

for i in range(time_step, len(scaled_test_data)):
    # time_step kadar geçmiş veriyi al
    X_test_single = scaled_test_data[i-time_step:i, 0]
    X_test_single = X_test_single.reshape(1, time_step, 1)

    # Tahmin yap
    predicted_price = model.predict(X_test_single)
    predicted_price = scaler.inverse_transform(predicted_price)

    # Gerçek fiyatı al
    actual_price = scaler.inverse_transform(scaled_test_data[i:i+1])[0, 0]

    # Karar ver
    if predicted_price > actual_price:
        decision = "BUY"
    elif predicted_price < actual_price:
        decision = "SELL"
    else:
        decision = "HOLD"

    # Sonuçları kaydet
    predictions.append(predicted_price[0, 0])
    decisions.append(decision)

# Tahmin sonuçlarını ve kararları DataFrame'e ekle
result = pd.DataFrame({
    'Actual': scaler.inverse_transform(scaled_test_data[time_step:]).flatten(),
    'Predicted': predictions,
    'Decision': decisions
})

# Sonuçları göster
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
       Actual   Predicted Decision
0  147.699997  150.948044      BUY
